# Введение в MapReduce модель на Python


In [2]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import random

In [3]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [4]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [5]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [6]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [7]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [8]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [9]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [10]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [11]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [12]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [13]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [14]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [16]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [18]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.8827674769292564)),
 (1, np.float64(2.8827674769292564)),
 (2, np.float64(2.8827674769292564)),
 (3, np.float64(2.8827674769292564)),
 (4, np.float64(2.8827674769292564))]

## Inverted index

In [19]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [20]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [21]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [22]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18)]),
 (1, [('is', 18), ('what', 10)])]

## TeraSort

In [23]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.01626737295636882)),
   (None, np.float64(0.11585406591169478)),
   (None, np.float64(0.1214420142843402)),
   (None, np.float64(0.12423770335568696)),
   (None, np.float64(0.1595890493536578)),
   (None, np.float64(0.1832002568741088)),
   (None, np.float64(0.18528468537618947)),
   (None, np.float64(0.2545452923463809)),
   (None, np.float64(0.26859711813658516)),
   (None, np.float64(0.2932575019461674)),
   (None, np.float64(0.37570038690844043)),
   (None, np.float64(0.40166135684573645)),
   (None, np.float64(0.4088492614534345)),
   (None, np.float64(0.4150344463353276)),
   (None, np.float64(0.43027865633404916)),
   (None, np.float64(0.45740574047458615)),
   (None, np.float64(0.47605108037017485))]),
 (1,
  [(None, np.float64(0.5328303073184525)),
   (None, np.float64(0.5483587043181393)),
   (None, np.float64(0.6015531968276581)),
   (None, np.float64(0.6210457981873215)),
   (None, np.float64(0.6721195145558766)),
   (None, np.float64(0.704191181

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [24]:
import random

def MAP(num_list):
    return max(num_list)


def REDUCE(num_list):
    return max(num_list)


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[25, 91, 37, 0, 35, 20, 26, 75, 30, 38, 91, 27, 56, 56, 71, 41, 81, 9, 49, 62, 50, 73, 60, 46, 94, 53, 18, 87, 28, 47, 63, 27, 7, 17, 24, 100, 28, 96, 18, 69, 19, 31, 50, 1, 76, 66, 11, 41, 26, 77, 21, 50, 63, 98, 80, 89, 52, 95, 90, 51, 15, 55, 59, 33, 26, 56, 64, 76, 4, 41, 16, 98, 3, 85, 30, 47, 93, 64, 2, 97, 43, 52, 23, 54, 14, 26, 74, 74, 62, 51, 78, 81, 27, 63, 2, 18, 33, 81, 79, 82]
[[25, 91, 37, 0, 35], [20, 26, 75, 30, 38], [91, 27, 56, 56, 71], [41, 81, 9, 49, 62], [50, 73, 60, 46, 94], [53, 18, 87, 28, 47], [63, 27, 7, 17, 24], [100, 28, 96, 18, 69], [19, 31, 50, 1, 76], [66, 11, 41, 26, 77], [21, 50, 63, 98, 80], [89, 52, 95, 90, 51], [15, 55, 59, 33, 26], [56, 64, 76, 4, 41], [16, 98, 3, 85, 30], [47, 93, 64, 2, 97], [43, 52, 23, 54, 14], [26, 74, 74, 62, 51], [78, 81, 27, 63, 2], [18, 33, 81, 79, 82]]
100


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [25]:
def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 44), (1, 76), (1, 49), (1, 83), (1, 73), (1, 79), (1, 15), (1, 83), (1, 86), (1, 41), (1, 80), (1, 45), (1, 16), (1, 1), (1, 40), (1, 23), (1, 36), (1, 64), (1, 82), (1, 41), (1, 39), (1, 89), (1, 86), (1, 95), (1, 92), (1, 80), (1, 4), (1, 18), (1, 73), (1, 69), (1, 42), (1, 93), (1, 1), (1, 23), (1, 18), (1, 55), (1, 91), (1, 37), (1, 23), (1, 35), (1, 82), (1, 16), (1, 63), (1, 25), (1, 15), (1, 92), (1, 0), (1, 3), (1, 27), (1, 82), (1, 31), (1, 71), (1, 15), (1, 46), (1, 69), (1, 27), (1, 86), (1, 39), (1, 54), (1, 23), (1, 13), (1, 61), (1, 68), (1, 54), (1, 23), (1, 22), (1, 22), (1, 77), (1, 97), (1, 48), (1, 82), (1, 96), (1, 33), (1, 3), (1, 4), (1, 8), (1, 96), (1, 0), (1, 69), (1, 97), (1, 61), (1, 15), (1, 70), (1, 67), (1, 66), (1, 14), (1, 75), (1, 38), (1, 21), (1, 10), (1, 12), (1, 11), (1, 66), (1, 91), (1, 14), (1, 71), (1, 9), (1, 75), (1, 25), (1, 39)]
[(1, [44, 76, 49, 83, 73, 79, 15, 83, 86, 41, 80, 45, 16, 1, 40, 23, 36, 64, 82, 41, 39, 89, 86, 95, 92, 80, 

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [26]:
def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 84), (1, 96), (1, 30), (1, 59), (1, 60), (1, 49), (1, 79), (1, 86), (1, 20), (1, 99), (1, 23), (1, 31), (1, 73), (1, 51), (1, 64), (1, 55), (1, 66), (1, 53), (1, 77), (1, 22), (1, 35), (1, 15), (1, 54), (1, 40), (1, 36), (1, 50), (1, 87), (1, 68), (1, 61), (1, 62), (1, 70), (1, 83), (1, 93), (1, 43), (1, 66), (1, 23), (1, 64), (1, 1), (1, 2), (1, 71), (1, 3), (1, 17), (1, 89), (1, 12), (1, 100), (1, 76), (1, 76), (1, 68), (1, 18), (1, 35), (1, 20), (1, 69), (1, 47), (1, 89), (1, 74), (1, 54), (1, 85), (1, 6), (1, 32), (1, 54), (1, 22), (1, 49), (1, 75), (1, 70), (1, 1), (1, 12), (1, 40), (1, 100), (1, 52), (1, 3), (1, 70), (1, 46), (1, 60), (1, 16), (1, 51), (1, 58), (1, 36), (1, 62), (1, 90), (1, 61), (1, 98), (1, 93), (1, 86), (1, 36), (1, 20), (1, 82), (1, 36), (1, 20), (1, 71), (1, 85), (1, 90), (1, 6), (1, 18), (1, 75), (1, 91), (1, 6), (1, 15), (1, 20), (1, 0), (1, 70)]
[(1, [84, 96, 30, 59, 60, 49, 79, 86, 20, 99, 23, 31, 73, 51, 64, 55, 66, 53, 77, 22, 35, 15, 54, 40, 36, 

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [27]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


# ===================================================================================================================================================================================

from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'a', 'banana', 'it']), (1, ['is', 'what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [28]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Выбираем только кортежи с четным первым элементом


def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[(85, 5, 86), (99, 52, 85), (20, 17, 41), (3, 15, 92), (7, 88, 22), (48, 95, 14), (30, 88, 81), (97, 91, 26), (62, 14, 73), (43, 38, 80), (88, 66, 13), (94, 62, 47), (0, 37, 47), (71, 98, 13), (91, 93, 67), (8, 19, 2), (46, 33, 91), (88, 28, 24), (0, 43, 66), (93, 74, 36), (84, 42, 96), (96, 60, 85), (94, 58, 54), (7, 1, 64), (52, 37, 75), (3, 1, 4), (7, 71, 50), (80, 71, 46), (66, 25, 76), (78, 92, 70), (82, 23, 87), (92, 81, 10), (86, 26, 13), (81, 5, 37), (7, 14, 99), (53, 11, 57), (90, 36, 18), (82, 28, 94), (43, 12, 60), (8, 41, 14), (94, 80, 10), (27, 4, 97), (76, 48, 65), (12, 0, 94), (70, 75, 45), (28, 65, 57), (76, 10, 39), (43, 90, 93), (94, 16, 83), (60, 44, 25), (63, 81, 61), (25, 64, 6), (10, 38, 44), (90, 76, 29), (0, 41, 22), (0, 85, 47), (52, 73, 57), (89, 80, 76), (64, 14, 0), (58, 5, 42), (82, 55, 0), (25, 29, 54), (73, 76, 44), (73, 74, 5), (23, 62, 68), (91, 100, 34), (38, 89, 15), (80, 85, 78), (10, 86, 84), (71, 19, 42), (39, 78, 44), (64, 71, 3), (77, 88, 17), (4

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [29]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(34, 5, 99), (98, 37, 81), (97, 1, 83), (43, 92, 57), (96, 77, 62), (7, 6, 12), (82, 97, 100), (73, 70, 100), (10, 68, 84), (85, 55, 63), (38, 62, 76), (85, 54, 92), (12, 41, 46), (38, 79, 20), (58, 28, 97), (18, 26, 28), (100, 19, 37), (52, 96, 31), (53, 45, 12), (93, 45, 51), (20, 86, 23), (78, 82, 27), (28, 59, 39), (27, 68, 39), (86, 20, 35), (80, 14, 20), (92, 40, 73), (24, 50, 28), (93, 11, 17), (47, 100, 98), (86, 58, 5), (0, 56, 39), (13, 17, 47), (19, 6, 87), (3, 47, 55), (54, 1, 21), (80, 28, 97), (50, 44, 45), (28, 62, 31), (17, 32, 19), (92, 1, 63), (7, 83, 11), (2, 31, 99), (16, 48, 61), (33, 17, 0), (50, 49, 38), (79, 59, 58), (37, 43, 7), (23, 26, 5), (55, 23, 79), (3, 36, 9), (7, 3, 1), (11, 50, 9), (18, 64, 15), (99, 11, 94), (32, 76, 14), (27, 47, 57), (40, 39, 21), (47, 72, 98), (41, 82, 43), (40, 57, 3), (98, 73, 98), (84, 77, 92), (22, 87, 93), (36, 55, 15), (44, 74, 73), (18, 34, 51), (76, 24, 80), (50, 16, 46), (43, 99, 97), (19, 12, 8), (55, 96, 71), (81, 23, 3

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [31]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(73, 52, 9), (96, 27, 19), (80, 84, 94), (79, 33, 15), (31, 35, 69), (51, 72, 90), (65, 79, 19), (62, 56, 96), (95, 25, 55), (5, 90, 60), (49, 18, 31), (18, 86, 35), (23, 39, 49), (47, 75, 32), (77, 64, 66), (81, 84, 4), (31, 100, 5), (70, 26, 100), (38, 30, 42), (62, 99, 35), (79, 96, 92), (96, 10, 73), (97, 88, 31), (68, 28, 84), (87, 14, 99), (81, 43, 18), (88, 63, 73), (55, 90, 9), (94, 22, 27), (90, 16, 94), (98, 54, 41), (36, 33, 9), (91, 98, 94), (94, 54, 90), (32, 68, 1), (10, 20, 64), (9, 56, 17), (13, 45, 77), (98, 40, 79), (20, 60, 4), (80, 19, 95), (12, 55, 59), (31, 56, 48), (22, 26, 43), (60, 52, 19), (16, 51, 73), (57, 23, 60), (36, 51, 98), (62, 11, 85), (21, 82, 100), (92, 9, 31), (79, 90, 89), (94, 44, 63), (61, 30, 90), (36, 27, 97), (31, 27, 67), (29, 72, 90), (68, 78, 10), (45, 10, 9), (90, 59, 44), (30, 8, 33), (42, 88, 13), (96, 80, 85), (96, 90, 40), (56, 62, 13), (42, 60, 64), (87, 53, 64), (99, 50, 15), (96, 66, 94), (91, 94, 75), (27, 37, 83), (46, 35, 7), (

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [32]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[(2, 2), (3, 0), (2, 0), (3, 1), (0, 2), (3, 2), (2, 0), (0, 2), (0, 1), (2, 3), (1, 2), (2, 3), (0, 3), (0, 0), (2, 0), (0, 0), (2, 2), (0, 3), (2, 2), (2, 1), (0, 0), (1, 3), (2, 1), (3, 1), (1, 1), (0, 3), (3, 0), (2, 0), (2, 1), (2, 3), (1, 0), (1, 3), (1, 3), (2, 2), (0, 2), (1, 3), (0, 1), (1, 2), (0, 0), (3, 0), (3, 2), (1, 2), (0, 1), (0, 0), (3, 1), (2, 3), (0, 3), (1, 0), (0, 1), (0, 3), (1, 0), (1, 3), (3, 2), (1, 2), (3, 3), (3, 0), (0, 1), (1, 2), (2, 1), (3, 3), (0, 2), (1, 2), (2, 0), (0, 1), (0, 3), (0, 1), (3, 2), (3, 1), (0, 2), (0, 3), (0, 3), (3, 0), (0, 1), (3, 3), (3, 3), (3, 3), (3, 0), (1, 3), (1, 3), (0, 2), (0, 3), (3, 2), (3, 0), (2, 1), (0, 0), (1, 1), (3, 3), (0, 2), (1, 3), (1, 0), (2, 2), (2, 2), (3, 3), (1, 1), (2, 0), (2, 1), (3, 3), (2, 2), (1, 0), (2, 3)]
[((2, 0), (2, 0)), ((3, 1), (3, 1)), ((0, 1), (0, 1)), ((1, 2), (1, 2)), ((2, 0), (2, 0)), ((1, 0), (1, 0)), ((1, 2), (1, 2)), ((2, 1), (2, 1)), ((2, 3), (2, 3)), ((1, 3), (1, 3)), ((0, 3), (0, 3)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [33]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]: # если соответствующее значение является списком  [R] , создайте пару  (t,t)
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[<__main__.Tuple object at 0x000001FAB6D83B00>, <__main__.Tuple object at 0x000001FAB6D95AC0>, <__main__.Tuple object at 0x000001FAB6D957F0>, <__main__.Tuple object at 0x000001FAB6D94AD0>, <__main__.Tuple object at 0x000001FAB6D95760>, <__main__.Tuple object at 0x000001FAB6D95F40>, <__main__.Tuple object at 0x000001FAB6D96180>, <__main__.Tuple object at 0x000001FAB6D96150>, <__main__.Tuple object at 0x000001FAB6D953D0>, <__main__.Tuple object at 0x000001FAB6D94740>, <__main__.Tuple object at 0x000001FAB6D97B90>, <__main__.Tuple object at 0x000001FAB6D95A30>, <__main__.Tuple object at 0x000001FAB6D95E80>, <__main__.Tuple object at 0x000001FAB6D95070>, <__main__.Tuple object at 0x000001FAB6D95640>, <__main__.Tuple object at 0x000001FAB6D978C0>, <__main__.Tuple object at 0x000001FAB6D97B30>, <__main__.Tuple object at 0x000001FAB6D94110>, <__main__.Tuple object at 0x000001FAB6D95E20>, <__main__.Tuple object at 0x000001FAB6D97F80>, <__main__.Tuple object at 0x000001FAB6D95B50>, <__main__.Tu

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [34]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[<__main__.Tuple object at 0x000001FAAC39B980>, <__main__.Tuple object at 0x000001FAABECC4A0>, <__main__.Tuple object at 0x000001FAB6D81310>, <__main__.Tuple object at 0x000001FAB6D83E00>, <__main__.Tuple object at 0x000001FAB6D94830>, <__main__.Tuple object at 0x000001FAB6D940B0>, <__main__.Tuple object at 0x000001FAB6D95460>, <__main__.Tuple object at 0x000001FAB6D942F0>, <__main__.Tuple object at 0x000001FAB6D97C20>, <__main__.Tuple object at 0x000001FAB6D97890>, <__main__.Tuple object at 0x000001FAB6D973B0>, <__main__.Tuple object at 0x000001FAB6D977D0>, <__main__.Tuple object at 0x000001FAB6D97650>, <__main__.Tuple object at 0x000001FAB6D974D0>, <__main__.Tuple object at 0x000001FAB6D974A0>, <__main__.Tuple object at 0x000001FAB6D97470>, <__main__.Tuple object at 0x000001FAB6D961E0>, <__main__.Tuple object at 0x000001FAB6D960C0>, <__main__.Tuple object at 0x000001FAB6D95BB0>, <__main__.Tuple object at 0x000001FAB6D95A60>, <__main__.Tuple object at 0x000001FAB6D95880>, <__main__.Tu

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [35]:
def get_random_tuple():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)


def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(2, 1, 1), (2, 2, 0), (1, 0, 1), (3, 2, 2), (3, 3, 3), (0, 0, 1), (3, 0, 1), (3, 2, 1), (1, 0, 0), (1, 1, 2), (1, 3, 2), (3, 3, 1), (3, 2, 1), (2, 0, 2), (1, 1, 1), (0, 2, 1), (0, 3, 3), (3, 2, 0), (3, 3, 3), (2, 2, 2), (1, 2, 2), (3, 3, 1), (1, 3, 3), (2, 3, 2), (0, 2, 0), (1, 1, 1), (0, 0, 1), (0, 2, 2), (2, 1, 0), (1, 2, 2), (3, 3, 0), (1, 0, 1), (0, 2, 2), (1, 2, 3), (3, 0, 0), (3, 0, 3), (3, 1, 3), (3, 2, 1), (1, 2, 1), (1, 1, 2), (3, 2, 0), (3, 2, 2), (3, 1, 3), (2, 2, 3), (0, 0, 0), (3, 3, 3), (0, 0, 2), (2, 2, 1), (2, 1, 0), (3, 0, 2), (3, 1, 2), (1, 3, 2), (0, 1, 0), (1, 3, 1), (3, 3, 2), (3, 3, 0), (1, 2, 1), (3, 3, 3), (3, 2, 1), (1, 1, 2), (3, 3, 0), (3, 0, 1), (1, 3, 0), (3, 0, 1), (0, 2, 3), (0, 1, 2), (3, 2, 0), (2, 3, 2), (1, 2, 2), (1, 3, 0), (2, 2, 1), (2, 3, 2), (0, 2, 2), (0, 3, 2), (2, 3, 0), (1, 1, 1), (0, 1, 2), (2, 3, 0), (2, 3, 1), (1, 0, 1), (3, 3, 1), (3, 2, 0), (2, 0, 3), (3, 1, 1), (0, 0, 3), (1, 2, 1), (0, 2, 2), (3, 1, 0), (2, 0, 3), (0, 0, 3), (3, 1, 2)

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [36]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [37]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [38]:
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) 

True

In [39]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [40]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j]) 

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k]) 

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)


def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)


def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) 
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))
  yield second_mat


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.013998545059063125), ((0, 1), 0.006146411005643418), ((0, 2), 0.0013094070280139968), ((0, 3), 0.006761229708196888), ((0, 4), 0.001640529289576924), ((0, 5), 0.002799185187369619), ((0, 6), 0.0105191523833935), ((0, 7), 0.0013711064884254377), ((0, 8), 0.013411951239838366), ((0, 9), 0.005426143970943269), ((0, 10), 0.007609049702543101), ((0, 11), 0.006972608712276354), ((0, 12), 0.003454543044118363), ((0, 13), 0.013734923700417226), ((0, 14), 0.00011522547046654885), ((0, 15), 0.009908856673628218), ((0, 16), 0.012300277275695567), ((0, 17), 0.013799111322498551), ((0, 18), 0.013437119711956177), ((0, 19), 0.0017433486286859733), ((0, 20), 0.0010128292848794714), ((0, 21), 0.004086297228970294), ((0, 22), 0.0003102038136482358), ((0, 23), 0.005563291398424726), ((0, 24), 0.0005735526644171284), ((0, 25), 0.01207463764694144), ((0, 26), 0.011518737208141352), ((0, 27), 0.010662534328019476), ((0, 28), 0.00844885799778191

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [41]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat


import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) 

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.44125258078026425)), ((0, 1), np.float64(0.651581671572432)), ((0, 2), np.float64(0.3952073477344883)), ((0, 3), np.float64(0.5698223977065272)), ((0, 4), np.float64(0.7035208429770731)), ((0, 5), np.float64(0.6099148000344784)), ((0, 6), np.float64(0.29792280006839617)), ((0, 7), np.float64(0.22638412206657885)), ((0, 8), np.float64(0.5405467840224453)), ((0, 9), np.float64(0.6558352155608952)), ((0, 10), np.float64(0.08260557120286942)), ((0, 11), np.float64(0.6711968695762418)), ((0, 12), np.float64(0.27022171230561176)), ((0, 13), np.float64(0.49347098460889915)), ((0, 14), np.float64(0.10521511641151265)), ((0, 15), np.float64(0.30482855769675626)), ((0, 16), np.float64(0.2157718664309384)), ((0, 17), np.float64(0.5185461702274509)), ((0, 18), np.float64(0.5975382797496663)), ((0, 19), np.float64(0.6955713288390437)), ((0, 20), np.float64(0.4846803388435306)), ((0, 21), np.float64(0.7012950431856656)), ((0, 

True